# Create a Credential file using Python

Source: https://www.geeksforgeeks.org/create-a-credential-file-using-python/

## Expiry file

### expiry.py
```
import os
import time

key_file = 'key.key'
key_exp_start = time.time()
cred_filename = 'CredFile.ini'

with open(cred_filename, 'r') as cred_in:

    lines = cred_in.readlines()
    config = {}
    for line in lines:
        tuples = line.rstrip('\n').split('=', 1)

        if tuples[0] in ('Expiry '):
            config[tuples[0]] = tuples[1]

    if not(config['Expiry '] == -1):

        # Time below is in seconds.
        time_for_exp = int(config['Expiry ']) * 60

        while(os.path.isfile(key_file)):
            time.sleep(10)

            if (not(time.time() - key_exp_start <= time_for_exp)
                and os.path.isfile(key_file)):
                os.remove(key_file)
```

## Create a credential file

In [5]:
# Creates a credential file
from cryptography.fernet import Fernet
import re
import ctypes
import time
import os
import sys

class Credentials():

    def __init__(self):
        self.__username = ""
        self.__key = ""
        self.__password = ""
        self.__key_file = 'key.key'
        self.__time_of_exp = -1

# Getter setter for attributes

    @property
    def username(self):
        return self.__username

    @username.setter
    def username(self,username):
        while (username == ''):
            username = input('Enter a proper User name, blank is not accepted:')
        self.__username = username

    @property
    def password(self):
        return self.__password

    @password.setter
    def password(self,password):
        self.__key = Fernet.generate_key()
        f = Fernet(self.__key)
        self.__password = f.encrypt(password.encode()).decode()
        del f

    @property
    def expiry_time(self):
        return self.__time_of_exp

    @expiry_time.setter
    def expiry_time(self,exp_time):
        if(exp_time >= 2):
            self.__time_of_exp = exp_time

            
    def create_cred(self):
        """
        This function is responsible for encrypting the password and create  key file for
        storing the key and create a credential file with user name and password
        """

        cred_filename = 'CredFile.ini'

        with open(cred_filename,'w') as file_in:
            file_in.write("#Credential file:\nUsername={}\nPassword={}\nExpiry={}\n".format(self.__username,self.__password,self.__time_of_exp)) 
            file_in.write("++"*20)
        
        # If there exists an older key file, this will remove it
        if(os.path.exists(self.__key_file)):
            os.remove(self.__key_file)

        # Open the Key.key file and place the key in it
        # The key file is hidden
        try:

            os_type = sys.platform
            if (os_type == 'linux'):
                self.__key_file = '.' + self.__key_file

            with open(self.__key_file,'w') as key_in:
                key_in.write(self.__key.decode())
                #Hidding the key file.
                #The below code snippet finds out which current os the scrip is running on and does the taks base on it.
                if(os_type == 'win32'):
                    ctypes.windll.kernel32.SetFileAttributesW(self.__key_file, 2)
                else:
                    pass

        except PermissionError:
            os.remove(self.__key_file)
            print("A Permission error occurred.\n Please re run the script")
            sys.exit()

        self.__username = ""
        self.__password = ""
        self.__key = ""
        self.__key_file
    

# Creating an object for Credentials class
creds = Credentials()

# Cred and Key files location
cred_filename = 'CredFile.ini'
file_location = str(os.path.realpath(cred_filename))
file_location = file_location.replace(os.getlogin(), "USERNAME")


#  Accepting credentials
creds.username = input("Enter UserName:")
creds.password = input("Enter Password:")
print("Enter the expiry time for key file in minutes, [default:Will never expire]")
creds.expiry_time = int(input("Enter time:") or '-1')

# Calling the Credit
creds.create_cred()
print("**"*20)
print("Cred file created successfully at {}".format(time.ctime()))
print("Cred file located at {}".format(file_location))

if not(creds.expiry_time == -1):
    os.startfile('expire.py')

print("**"*20)

Enter UserName:UserName
Enter Password:Pas$worD
Enter the expiry time for key file in minutes, [default:Will never expire]
Enter time:
****************************************
Cred file created successfully at Sun Jan 17 14:51:47 2021
Cred file located at /Users/USERNAME/LearningPython/PLAYING/Cryptography/CredFile.ini
****************************************


## Retrive credentials

In [2]:
# Retrive credentials
from cryptography.fernet import Fernet
import os

cred_filename = 'CredFile.ini'
key_file = 'key.key'

key = ''

with open('key.key','r') as key_in:
    key = key_in.read().encode()

#If you want the Cred file to be of one time use, uncomment the line below
#os.remove(key_file)

f = Fernet(key)
with open(cred_filename,'r') as cred_in:
    lines = cred_in.readlines()
    config = {}
    for line in lines:
        tuples = line.rstrip('\n').split('=',1)
        if tuples[0] in ('Username','Password'):
            config[tuples[0]] = tuples[1]

    passwd = f.decrypt(config['Password'].encode()).decode()
    print("Password:", passwd) 

Password: Pas$worD
